In [8]:
import requests
from PIL import Image
from io import BytesIO
from bs4 import BeautifulSoup
from transformers import AutoProcessor, BlipForConditionalGeneration

In [ ]:
processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

c:\Users\zeina\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
url = "https://en.wikipedia.org/wiki/IBM"

In [10]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [11]:
img_elements = soup.find_all('img')

In [12]:
with open("captions.txt", "w") as caption_file:
    for img_element in img_elements:
        img_url = img_element.get('src')
        if 'svg' in img_url or '1x1' in img_url:
            continue
        if img_url.startswith('//'):
            img_url = 'https:' + img_url
        elif not img_url.startswith('http://') and not img_url.startswith('https://'):
            continue
        try:
            response = requests.get(img_url)
            raw_image = Image.open(BytesIO(response.content))
            if raw_image.size[0] * raw_image.size[1] < 400:
                continue
            
            raw_image = raw_image.convert('RGB')
            inputs = processor(raw_image, return_tensors="pt")
            out = model.generate(**inputs, max_new_tokens=50)
            caption = processor.decode(out[0], skip_special_tokens=True)
            caption_file.write(f"{img_url}: {caption}\n")
        except Exception as e:
            print(f"Error processing image {img_url}: {e}")
            continue

Error processing image https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/IBM_Q_system_%28Fraunhofer_2%29.jpg/220px-IBM_Q_system_%28Fraunhofer_2%29.jpg: cannot identify image file <_io.BytesIO object at 0x0000013F31C91850>
